In [1]:
import sys
import torchio as tio
import pandas as pd 
import numpy as np
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm


sys.path.insert(0, '../../scripts/')
import data_loader
from helpers import miscellaneous as misc

CONFIG = misc.get_config()

In [2]:
CONFIG

{'RAW_DATA_DIR': 'data/raw/',
 'PROCESSED_DATA_DIR': 'data/processed/',
 'FLATTENED_DATA_DIR': 'data/raw/flattened/',
 'SERIALIZED_MODELS': 'models/trained_models/',
 'METRICS_DIR': 'models/trained_history/',
 'ANNOTATIONS_DIR': 'data/annotations/',
 'CDR_DIR': 'data/tabular_data/tabular_data/CDR.csv',
 'GDSCALE_DIR': 'data/tabular_data/tabular_data/GDSCALE.csv',
 'MMSE_DIR': 'data/tabular_data/tabular_data/MMSE.csv',
 'MOCA_DIR': 'data/tabular_data/tabular_data/MOCA.csv',
 'NEUROBAT_DIR': 'data/tabular_data/tabular_data/NEUROBAT.csv',
 'PTDEMOG_DIR': 'data/tabular_data/tabular_data/PTDEMOG.csv',
 'TRAIN_LABELS_DIR': 'data/annotations/train_labels.csv',
 'TEST_LABELS_DIR': 'data/annotations/test_labels.csv',
 'PLOT_DIR_BINARY': 'reports/plots/',
 'SLICED_DATA_DIR': 'data/raw/slices/',
 'MODEL': 'DenseNet121',
 'DEVICE': 'gpu',
 'TRAIN_SIZE': 0.8,
 'LEARNING_RATE': 0.001,
 'AUTO_LEARNING_RATE': True,
 'BATCH_SIZE': 128,
 'AUTO_BATCH_SIZE': False,
 'EPOCHS': 30,
 'NUM_WORKERS': 5,
 'TRAN

In [3]:
SLICE = CONFIG['NSLICE']
DIMENSION = CONFIG['DIMENSION']

In [28]:
train_data = pd.read_csv('../../' + CONFIG['TRAIN_LABELS_DIR'])
test_data = pd.read_csv('../../' + CONFIG['TEST_LABELS_DIR'])

#TRANSFORMER = CONFIG['TRANSFORMER']
#train_transform, val_transform = prep.get_transformer(TRANSFORMER)

In [29]:
def imshow(img):
    plt.imshow(img)
    
def save_slices(annotations):
    
    for i in tqdm(range(len(annotations))):
        filename = annotations.filename.iloc[i]
        if isinstance(filename, str):
            name = filename.split('\\')[1]
            name = name.replace('.nii', '.png')
        else:
            continue

        img = tio.ScalarImage("../../" + filename)
        
        if SLICE == 'Center':
            nslice = img.data.size(DIMENSION)//2
        else:
            nslice = SLICE
        
        if SLICE:
            if DIMENSION == 1:
                assert img.data.size()[1] >= nslice, f'[WARN] Nslice is too large. Max allowed: {img.data.size()[1]}'
                img = img.data[:, nslice, :, :]
            elif DIMENSION == 2:
                assert img.data.size()[2] >= nslice, f'[WARN] Nslice is too large. Max allowed: {img.data.size()[2]}'
                img = img.data[:, :, nslice, :]
            elif DIMENSION == 3:
                assert img.data.size()[3] >= nslice, f'[WARN] Nslice is too large. Max allowed: {img.data.size()[3]}'
                img = img.data[:, :, :, nslice]
        
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        npimg = np.squeeze(npimg)
        im = Image.fromarray(npimg).convert("L")
        
        im.save("../../" + CONFIG['SLICED_DATA_DIR'] + f'center/{DIMENSION}/' + name)

    annotations.filename = annotations.filename.str.replace('.nii', '.png')
    annotations.filename = annotations.filename.str.replace('/flattened', '/slices/' + f'center/{DIMENSION}')
    return annotations
# trainset
new_train_annotations = save_slices(train_data)
new_train_annotations.to_csv('../../' + CONFIG['TRAIN_LABELS_DIR'], mode='w+')
new_test_annotations = save_slices(test_data)
new_test_annotations.to_csv('../../' + CONFIG['TEST_LABELS_DIR'], mode='w+')

  0%|          | 0/1824 [00:00<?, ?it/s]

C:\Users\bruno\AppData\Local\Temp\ipykernel_11172\1650925712.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  annotations.filename = annotations.filename.str.replace('.nii', '.png')


  0%|          | 0/470 [00:00<?, ?it/s]

In [7]:
new_train_annotations

,filename,Group,Subject
0,data/raw/flattened\ADNI_941_S_1311_MR_MPR__Gra...,MCI,941_S_1311
1,data/raw/flattened\ADNI_941_S_1311_MR_MPR__Gra...,MCI,941_S_1311
2,data/raw/flattened\ADNI_941_S_1311_MR_MPR-R__G...,MCI,941_S_1311
3,data/raw/flattened\ADNI_141_S_1255_MR_MPR__Gra...,MCI,141_S_1255
4,data/raw/flattened\ADNI_141_S_1255_MR_MPR__Gra...,MCI,141_S_1255
...,...,...,...
1819,data/raw/flattened\ADNI_002_S_0816_MR_MPR__Gra...,AD,002_S_0816
1820,data/raw/flattened\ADNI_002_S_0619_MR_MPR__Gra...,AD,002_S_0619
1821,data/raw/flattened\ADNI_002_S_0619_MR_MPR-R__G...,AD,002_S_0619
1822,data/raw/flattened\ADNI_002_S_0619_MR_MPR-R__G...,AD,002_S_0619


In [64]:
from data_loader import get_data_loader

dataloader = get_data_loader()
train_dataloader = dataloader('../../' + CONFIG['TRAIN_LABELS_DIR'], transform=train_transform, dimension=CONFIG['DIMENSION'], nslice=CONFIG['NSLICE'])

train_dataloader[9]

ValueError: The input must be a 4D tensor with dimensions (channels, x, y, z) but it has shape (1, 166, 1). Tips: if it is a volume, please add the channels dimension; if it is 2D, also add a dimension of size 1 for the z axis